In [1]:
import numpy as np

from keras.models import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from models.unet import *
from models.unet_se import *

from datahandler import DataHandler
from data_loader import *
from params import *
import os
import cv2
import skimage.io as io
from tqdm import tqdm
from math import ceil
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, jaccard_similarity_score

from scipy.ndimage import _ni_support
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion,\
     generate_binary_structure

import warnings
warnings.filterwarnings("ignore")

plt.gray()

Using TensorFlow backend.


In [2]:
def getGenerator(images, bs=1):
    image_datagen = ImageDataGenerator(rescale=1./255)
    image_datagen.fit(images, augment = True)
    image_generator = image_datagen.flow(x = images, batch_size=bs,
            shuffle = False)

    return image_generator


In [3]:
def getDiceScore(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)


In [4]:
 def hd(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

def hd95(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity)
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity)
    hd95 = np.percentile(np.hstack((hd1, hd2)), 95)
    return hd95

def __surface_distances(result, reference, voxelspacing=None, connectivity=1):
    result = np.atleast_1d(result.astype(np.bool))
    reference = np.atleast_1d(reference.astype(np.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = np.asarray(voxelspacing, dtype=np.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()

    footprint = generate_binary_structure(result.ndim, connectivity)

    if 0 == np.count_nonzero(result):
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == np.count_nonzero(reference):
        raise RuntimeError('The second supplied array does not contain any binary object.')

    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)

    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]

    return sds

In [5]:
image_files, mask_files = load_data_files('data/kfold_data/')

skf = getKFolds(image_files, mask_files, n=5)

kfold_indices = []
for train_index, val_index in skf.split(image_files, mask_files):
    kfold_indices.append({'train': train_index, 'val': val_index})

In [6]:
def predictMask(model, image):  
    image_gen = getGenerator(image)
    return model.predict_generator(image_gen, steps=len(image))

In [7]:
def predictAll(model, data, num_data=0):
    dice_scores = []
    hd_scores = []
    hd95_scores = []

    for image_file, mask_file in tqdm(data, total=num_data):
        
        image, hdr = dh.getImageData(image_file)
        gt_mask, _ = dh.getImageData(mask_file, is_mask=True)

        assert image.shape == gt_mask.shape
        if image.shape[1] != 256:
            continue

        pred_mask = predictMask(model, image)
        pred_mask[pred_mask>=0.7] = 1
        pred_mask[pred_mask<0.7] = 0
            
        dice_score = getDiceScore(gt_mask, pred_mask)
        dice_scores.append(dice_score)
        
        hd_score = hd(gt_mask, pred_mask)
        hd_scores.append(hd_score)
        
        hd95_score = hd95(gt_mask, pred_mask)
        hd95_scores.append(hd95_score)

    return np.mean(dice_scores), np.mean(hd_scores), np.mean(hd95_scores)

In [8]:
#Get data and generators

unet_type = 'unet'
dh = DataHandler()
all_dice = []
all_hd = []
all_hd95 = []

for i in range(len(kfold_indices)):
    exp_name = 'kfold_unet_dice_DA_K%d'%i

    #get parameters
    params = getParams(exp_name, unet_type=unet_type)
    
    val_img_files = np.take(image_files, kfold_indices[i]['val'])
    val_mask_files = np.take(mask_files, kfold_indices[i]['val'])
    
    
    if unet_type == 'unet_se':
        model = getSEUnet()
        
    else:
        model = getUnet()
    
    print('loading weights from %s'%params['checkpoint']['name'])
    model.load_weights(params['checkpoint']['name'])
        
    data = zip(val_img_files, val_mask_files)
    
    dice_score, hd_score, hd95_score = predictAll(model, data, num_data=len(val_mask_files))
    
    print('K%d results'%i)
    print('dice %f'%dice_score)
    print('hd %f'%hd_score)
    print('hd95 %f'%hd95_score)
    
    
    all_dice.append(dice_score)
    all_hd.append(hd_score)
    all_hd95.append(hd95_score)
    
print('Final results for %s'%unet_type)
print('dice %f'%np.mean(all_dice))
print('hd %f'%np.mean(all_hd))
print('hd95 %f'%np.mean(all_hd95))

loading weights from ./logs/unet/kfold_unet/kfold_unet_dice_DA_K0/kfold_unet_dice_DA_K0_weights.h5


100%|██████████| 59/59 [04:20<00:00,  4.16s/it]


K0 results
dice 0.914345
hd 27.125448
hd95 3.295741


  0%|          | 0/58 [00:00<?, ?it/s]

loading weights from ./logs/unet/kfold_unet/kfold_unet_dice_DA_K1/kfold_unet_dice_DA_K1_weights.h5


...

K3 results
dice 0.928330
hd 22.327450
hd95 0.869211


  0%|          | 0/58 [00:00<?, ?it/s]

loading weights from ./logs/unet/kfold_unet/kfold_unet_dice_DA_K4/kfold_unet_dice_DA_K4_weights.h5


100%|██████████| 58/58 [04:17<00:00,  5.63s/it]

K4 results
dice 0.909402
hd 37.760048
hd95 6.508962
Final results for unet
dice 0.922734
hd 27.490035
hd95 2.802414


In [9]:
#Get data and generators

unet_type = 'unet_se'
dh = DataHandler()
all_dice = []
all_hd = []
all_hd95 = []

for i in range(len(kfold_indices)):
    exp_name = 'kfold_unet_dice_DA_K%d'%i

    #get parameters
    params = getParams(exp_name, unet_type=unet_type)
    
    val_img_files = np.take(image_files, kfold_indices[i]['val'])
    val_mask_files = np.take(mask_files, kfold_indices[i]['val'])
    
    
    if unet_type == 'unet_se':
        model = getSEUnet()
    
    print('loading weights from %s'%params['checkpoint']['name'])
    model.load_weights(params['checkpoint']['name'])
        
    data = zip(val_img_files, val_mask_files)
    
    dice_score, hd_score, hd95_score = predictAll(model, data, num_data=len(val_mask_files))
    
    print('K %d results'%i)
    print('dice %f'%dice_score)
    print('hd %f'%hd_score)
    print('hd95 %f'%hd95_score)
    
    
    all_dice.append(dice_score)
    all_hd.append(hd_score)
    all_hd95.append(hd95_score)
    
print('Final results for %s'%unet_type)
print('dice %f'%np.mean(all_dice))
print('hd %f'%np.mean(all_hd))
print('hd95 %f'%np.mean(all_hd95))

loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_dice_DA_K0/kfold_unet_dice_DA_K0_weights.h5


100%|██████████| 59/59 [04:45<00:00,  4.61s/it]


K 0 results
dice 0.934232
hd 26.334536
hd95 1.878845
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_dice_DA_K1/kfold_unet_dice_DA_K1_weights.h5


100%|██████████| 58/58 [04:24<00:00,  4.15s/it]


K 1 results
dice 0.937364
hd 25.393205
hd95 0.630005
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_dice_DA_K2/kfold_unet_dice_DA_K2_weights.h5


...

K 2 results
dice 0.940582
hd 14.540643
hd95 1.181931
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_dice_DA_K3/kfold_unet_dice_DA_K3_weights.h5


100%|██████████| 58/58 [04:31<00:00,  4.24s/it]


K 3 results
dice 0.931239
hd 35.582699
hd95 0.719255
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_dice_DA_K4/kfold_unet_dice_DA_K4_weights.h5


100%|██████████| 58/58 [04:40<00:00,  6.16s/it]

K 4 results
dice 0.927510
hd 29.151175
hd95 1.045047
Final results for unet_se
dice 0.934185
hd 26.200452
hd95 1.091017
